In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        pass
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
#랜덤시드 고정
import random
import torch

seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

In [3]:
#gpu 사용가능하면 사용하는 설정
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [4]:
#데이터 로드
sample = pd.read_csv('/kaggle/input/2022aifinalexamp4/submit.csv')

In [5]:
#학습데이터, 테스트데이터 경로 지정
from os.path import join
from glob import glob

train_path = '/kaggle/input/2022aifinalexamp4/train_data/train_images'
train_image_path = glob(join(train_path,'*','*'))
test_path = '/kaggle/input/2022aifinalexamp4/test_data/test_images'
test_image_path = glob(join(test_path,'*'))
test_sorted_path = []
for i in range(len(test_image_path)):
    test_sorted_path.append(test_path + '/' + str(i) + '.png')

In [6]:
#학습데이터셋, 테스트데이터셋 정의
from torch.utils.data import Dataset, DataLoader
from PIL import Image

class Datasets(Dataset):
    def __init__(self, path, state='',transform=None):
        self.state = state
        self.transform = transform
        self.path = path
        
        if self.state == 'train':
            self.label = [int(i.split('/')[-2]) for i in self.path]
    
    def __len__(self):
        return len(self.path)
    
    def __getitem__(self, index):
        data = self.path[index]
        data = Image.open(data)
        data = self.transform(data)
        
        if self.state == 'test':
            return data
        
        else:
            return data, torch.from_numpy(np.array(self.label[index]))

In [7]:
#데이터 정규화 방식
import torchvision.transforms as transforms

transform = transforms.Compose([
    transforms.Resize((227, 227)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (1.0,))
])

In [8]:
#학습데이터셋, 테스트데이터셋 설정
train_dataset = Datasets(train_image_path, state='train', transform=transform)
test_dataset = Datasets(test_sorted_path, state='test', transform=transform)

In [9]:
#학습데이터로더, 테스트데이터로더 설정
train_dataloader = DataLoader(train_dataset, batch_size=100,shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=100,shuffle=False)

In [10]:
#모델 정의
class Model(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = torch.nn.Conv2d(1, 96, 11, 4)
        self.max1 = torch.nn.MaxPool2d(3, 2)
        
        self.conv2 = torch.nn.Conv2d(96, 256, 5, 1)
        self.max2 = torch.nn.MaxPool2d(3, 2)
        
        self.conv3 = torch.nn.Conv2d(256, 384, 3, 1, 1)
        self.conv4 = torch.nn.Conv2d(384, 384, 3, 1, 1)

        self.conv5 = torch.nn.Conv2d(384, 256, 3, 1, 1)
        self.max3 = torch.nn.MaxPool2d(3, 2)
        
        self.linear1 = torch.nn.Linear(6400, 4096)
        self.linear2 = torch.nn.Linear(4096, 4096)
        self.linear3 = torch.nn.Linear(4096, 10)
        
        self.relu = torch.nn.ReLU()
        self.dropout = torch.nn.Dropout(0.5)
        
        torch.nn.init.xavier_uniform_(self.linear1.weight)
        torch.nn.init.xavier_uniform_(self.linear2.weight)
        torch.nn.init.xavier_uniform_(self.linear3.weight)
        torch.nn.init.xavier_uniform_(self.conv1.weight)
        torch.nn.init.xavier_uniform_(self.conv2.weight)
        torch.nn.init.xavier_uniform_(self.conv3.weight)
        torch.nn.init.xavier_uniform_(self.conv4.weight)
        torch.nn.init.xavier_uniform_(self.conv5.weight)
        
    def forward(self, x):
        x = self.max1(self.relu(self.conv1(x)))
        x = self.max2(self.relu(self.conv2(x)))
        x = self.relu(self.conv4(self.relu(self.conv3(x))))
        x = self.max3(self.relu(self.conv5(x)))
        x = x.view(x.size(0), -1)
        x = self.linear1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.linear2(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.linear3(x)
        
        return x

In [11]:
#모델 선언
model = Model().to(device)

In [12]:
#optimizer 정의
optimizer = torch.optim.Adam(model.parameters(),lr=1e-3)
#loss 정의 (다중분류이므로 CrossEntropyLoss)
loss = torch.nn.CrossEntropyLoss()

In [13]:
#모델 학습
from tqdm import tqdm

epochs = 5
model.train()
for epoch in range(epochs):
    costs = []
    for x, y in tqdm(train_dataloader):
        x = x.to(device)
        y = y.to(device)
        
        optimizer.zero_grad()
        hypothesis = model(x)
        cost = loss(hypothesis, y)
        costs.append(cost.item())
        cost.backward()
        optimizer.step()
    
    average = np.mean(costs)
    print(f"epoch : {epoch}, cost : {average}")

100%|██████████| 600/600 [04:25<00:00,  2.26it/s]


epoch : 0, cost : 0.6005554352204004


100%|██████████| 600/600 [01:48<00:00,  5.55it/s]


epoch : 1, cost : 0.34946644665052495


100%|██████████| 600/600 [01:48<00:00,  5.52it/s]


epoch : 2, cost : 0.3052241962527235


100%|██████████| 600/600 [01:47<00:00,  5.58it/s]


epoch : 3, cost : 0.27656872898340223


100%|██████████| 600/600 [01:47<00:00,  5.59it/s]

epoch : 4, cost : 0.2613428446277976


In [14]:
#테스트 데이터를 이용해서 모델 평가
model.eval()
with torch.no_grad():
    predicts = []
    for x in tqdm(test_dataloader):
        x = x.to(device)
        hypothesis = model(x)
        predict = torch.argmax(hypothesis, dim=1)
        predicts.extend(predict.cpu().detach().numpy())

100%|██████████| 100/100 [00:35<00:00,  2.79it/s]


In [15]:
#결과값을 csv 파일로 변환
sample['label'] = predicts
sample.to_csv('submit.csv',index=False)